In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from tqdm import tqdm
import numpy as np
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

## Hyperparameters and Model Architecture

In [3]:
WEIGHT_DECAY = 0.0
DROPOUT_RATE = 0.2
LEARNING_RATE = 0.001
BATCH_SIZE = 10
CRITERION = F.nll_loss
EPOCHS = 3

In [7]:
class CNN(nn.Module):
    def __init__(self, y: int, z: int, convolution_function: nn.Module):
        super(CNN, self).__init__() 
        self.dropout = nn.Dropout(DROPOUT_RATE)
        self.conv1 = convolution_function(z, 32, 3)
        self.conv2 = convolution_function(32, 64, 3)
        self.conv3 = convolution_function(64, 128, 3)
        x = torch.randn(z, y, y).view(-1, z, y, y)
        self.to_linear = None
        self.convs(x)
        self.fc1 = nn.Linear(self.to_linear, 512)
        self.fc2 = nn.Linear(512, 10)

    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
        if self.to_linear is None:
            self.to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
        return x

    def forward(self, x):
        x = F.pad(x, (0, 0, 0, 0))
        x = self.convs(x)
        x = x.view(-1, self.to_linear)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### MNIST

In [5]:
train = datasets.MNIST("./347data", train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
test = datasets.MNIST("./347data", train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))
validation_set_size = int(len(train) * 0.1)
training_set_size = len(train) - validation_set_size
train_set, validation_set = torch.utils.data.random_split(train, [training_set_size, validation_set_size])
train_set = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
validation_set = torch.utils.data.DataLoader(validation_set, batch_size=BATCH_SIZE, shuffle=True)
test_set = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE, shuffle=True)

In [8]:
MNIST_net = CNN(28, 1, nn.Conv2d).cuda()
optimizer = optim.Adam(MNIST_net.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
for batch in range(EPOCHS):
    for data in tqdm(train_set):
        X, y = data
        MNIST_net.zero_grad()
        output = MNIST_net(X.cuda())
        loss = CRITERION(output, y.cuda())
        loss.backward()
        optimizer.step()
    print(loss)

  0%|          | 13/6000 [00:00<00:48, 123.67it/s]

tensor(0.0405, device='cuda:0', grad_fn=<NllLossBackward0>)


  0%|          | 22/6000 [00:00<00:27, 215.23it/s]

tensor(0.0054, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████| 6000/6000 [00:34<00:00, 176.34it/s]

tensor(0.0004, device='cuda:0', grad_fn=<NllLossBackward0>)


In [9]:
output = []
true = []
MNIST_net.eval()
with torch.no_grad():
    for data in validation_set:
        X, y = data
        for i in MNIST_net(X.cuda()):
            output.append(torch.argmax(i).cpu())
        for i in y:
            true.append(i)
MNIST_net.train()
print("Validation Accuracy:", metrics.accuracy_score(true, output))
print("Validation F1 Score:", metrics.f1_score(true, output, average="macro"))
true = np.eye(10)[true]
output = np.eye(10)[output]
print("Validation AUC Score:", metrics.roc_auc_score(true, output, multi_class="ovo", average="macro"))
            

Validation Accuracy: 0.993
Validation F1 Score: 0.992990445536965
Validation AUC Score: 0.996131815536789


### CIFAR-10

In [10]:
train = datasets.CIFAR10("./347data", train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
test = datasets.CIFAR10("./347data", train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))
validation_set_size = int(len(train) * 0.1)
training_set_size = len(train) - validation_set_size
train_set, validation_set = torch.utils.data.random_split(train, [training_set_size, validation_set_size])
train_set = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
validation_set = torch.utils.data.DataLoader(validation_set, batch_size=BATCH_SIZE, shuffle=True)
test_set = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
CIFAR10_net = CNN(32, 3, nn.Conv2d).cuda()
optimizer = optim.Adam(CIFAR10_net.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
for batch in range(EPOCHS):
    for data in tqdm(train_set):
        X, y = data
        CIFAR10_net.zero_grad()
        output = CIFAR10_net(X.cuda())
        loss = CRITERION(output, y.cuda())
        loss.backward()
        optimizer.step()
    print(loss)

  0%|          | 15/5000 [00:00<00:35, 141.43it/s]

tensor(1.6725, device='cuda:0', grad_fn=<NllLossBackward0>)


  0%|          | 16/5000 [00:00<00:31, 156.23it/s]

tensor(0.9463, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████| 5000/5000 [00:31<00:00, 159.27it/s]

tensor(0.9590, device='cuda:0', grad_fn=<NllLossBackward0>)


In [12]:
output = []
true = []
CIFAR10_net.eval()
with torch.no_grad():
    for data in validation_set:
        X, y = data
        for i in CIFAR10_net(X.cuda()):
            output.append(torch.argmax(i).cpu())
        for i in y:
            true.append(i)
CIFAR10_net.train()
print("Validation Accuracy:", metrics.accuracy_score(true, output))
print("Validation F1 Score:", metrics.f1_score(true, output, average="macro"))
true = np.eye(10)[true]
output = np.eye(10)[output]
print("Validation AUC Score:", metrics.roc_auc_score(true, output, multi_class="ovo", average="macro"))
            

Validation Accuracy: 0.6664
Validation F1 Score: 0.6557049535746451
Validation AUC Score: 0.815043988147967


### Iyer

In [14]:
iyer = np.loadtxt(open("347data/iyer.txt", "rb"), delimiter="\t")
X = iyer[:, 2:]
y = iyer[:, 1]
# print(X.shape)
test_set_size = int(len(X) * 0.1)
training_set_size = len(X) - test_set_size
validation_set_size = int(training_set_size * 0.1)
training_set_size -= validation_set_size
one_hot_y = [np.eye(12)[int(i)] for i in y]
data = [X, one_hot_y]
training = []  
test = []
validation = []
for i in range(len(data[0])):
    if i < training_set_size:
        training.append([data[0][i], data[1][i]])
    elif i < training_set_size + validation_set_size:
        validation.append([data[0][i], data[1][i]])
    else:
        test.append([data[0][i], data[1][i]])
tensor_train = torch.Tensor(np.array(training))
tensor_validation = torch.Tensor(np.array(validation))
tensor_test = torch.Tensor(np.array(test))
train_set = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
validation_set = torch.utils.data.DataLoader(validation_set, batch_size=BATCH_SIZE, shuffle=True)
test_set = torch.utils.data.DataLoader(tensor_test, batch_size=BATCH_SIZE, shuffle=True)

In [75]:
print(validation_set)
Iyer_net = CNN(1, 12, nn.Conv1d).cuda()
optimizer = optim.Adam(Iyer_net.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
EPOCHS = 3
for batch in range(EPOCHS):
    for data in tqdm(train_set):
        X, y = data
        Iyer_net.zero_grad()
        output = Iyer_net(X.cuda())
        loss = CRITERION(output, y.cuda())
        loss.backward()
        optimizer.step()
    print(loss)

RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (3 x 3). Kernel size can't be greater than actual input size